In [1]:
%%writefile Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

from enum import Enum
import warnings
import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju


# __cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_dir__ = 'C://cache.feather/'
__file_type__ = '.feather'

class CACHE_TYPE(Enum):
    default = ''
    STOCK = '/stock/'
    FACTOR = '/factor/'
    MARKET = '/market/'
    TMP = '/tmp/'
    
# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name, cache_type=CACHE_TYPE.default):
    # print(__cache_dir__ + cache_type.value + file_name + __file_type__)
    return __cache_dir__ + cache_type.value + file_name + __file_type__

def is_cache_exist(cache_name, cache_type=CACHE_TYPE.default):
    return os.path.exists(file_path(cache_name,cache_type))

def save_cache(name, data, cache_type=CACHE_TYPE.default):
    data.reset_index().to_feather(file_path(name,cache_type=cache_type))
    
def load_cache(cache_name:str, to_series:bool=False, time_flag:str='date',cache_type=CACHE_TYPE.default):
    if not is_cache_exist(cache_name,cache_type=cache_type):
        warnings.warn('{} cache not exist, create first'.format(cache_name)) 
        return None
        
    df = pd.read_feather(file_path(cache_name, cache_type=cache_type)).set_index([time_flag,'code']).sort_index(level=0)
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(cache_name:str, start:str, end:str, to_series:bool=False, time_flag:str='date',cache_type=CACHE_TYPE.default):
    if not is_cache_exist(cache_name,cache_type=cache_type):
        warnings.warn('{} cache not exist, create first'.format(cache_name)) 
        return None
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(cache_name, cache_type=cache_type)).set_index([time_flag,'code']).sort_index(level=0)
    dt_index = df.index.get_level_values(0)
    date_range = dt_index.unique()
    if not (st >= date_range.min() and en <= date_range.max()):
        info = '"{}"：date range out of cache[{},{}]'.format(
            cache_name,
            date_range.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_range.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        warnings.warn(info)
        
    df = df.loc[(dt_index >= start) & (dt_index <= end)]

    if to_series:
        df =  df.squeeze()
    return df


def load_caches_adv(cache_names:list, start:str=None, end:str=None, time_flag:str='date',cache_type=CACHE_TYPE.default):
    if not start is None:
        assert not end is None, 'start and end must not be None at the same time,or be None at the same time'
    assert isinstance(cache_names,list), 'cache_names MUST be list'
    
    tmp = None
    for name in cache_names:
        if start is None:
            df = load_cache(name,cache_type=cache_type)
        else:
            df = load_cache_adv(name,start,end,time_flag=time_flag,cache_type=cache_type)
        tmp = pd.concat([tmp,df],axis=1) 
        
    return tmp.sort_index()
     
    
        

Overwriting Cacher.py


In [17]:
print(__cache_dir__  +  CACHE_TYPE.STOCK.value + "file_name" + __file_type__ )

C://cache.feather/STOCKfile_name.feather


In [164]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()
# load_cache_adv('test','2011-11-11','2022-11-11')
load_caches_adv(['alpha11','alpha24'],'2011-11-11','2020-11-11')

alpha11  alpha24
date       code                     
2011-11-11 000001  0.468002     0.17
           000002  0.134449     0.16
           000063  0.301422     0.19
           000066  1.043020    -0.44
           000069  0.399679     0.15
...                     ...      ...
2020-11-11 603882  1.653110    15.08
           603899  0.482447    -2.20
           603939  1.360609     1.28
           603986  1.024721   -28.90
           603993  0.146505     0.04

[449688 rows x 2 columns]